### CYCLONE IDAI : Mozambique SAR Flood Mapping Approach 
  
Within this script SAR Sentinel-1 is being used to generate a flood extent map. Flood mapping using a change detection methodology with Copernicus S1. Ground Range Detected imagery includes the following   preprocessing steps: Thermal-Noise Removal, Radiometric calibration,  Terrain-correction  hence only a Speckle filter needs to be applied in the preprocessing.  
  
This script is based on the recommendations and demonstrations script located at:
https://un-spider.org/advisory-support/recommended-practices/recommended-practice-google-earth-engine-flood-mapping/step-by-step#Step%202:%20Time%20frame%20and%20sensor%20parameters%20selection

In [137]:
import ee
import folium

ee.Initialize()

In [138]:
# Get input datasets'

# country outline
aoi = ee.FeatureCollection("FAO/GAUL/2015/level0").filterMetadata('ADM0_NAME', 'equals', 'Mozambique')
sar = ee.ImageCollection("COPERNICUS/S1_GRD")
WorldPop = ee.ImageCollection("WorldPop/GP/100m/pop")
surfwater = ee.Image('JRC/GSW1_3/GlobalSurfaceWater').select('seasonality')
DEM = ee.Image('WWF/HydroSHEDS/03VFDEM')

In [139]:
# Window for images before flooding
before_start= '2019-01-01'
before_end='2019-02-28'

# Dates for flooding period
after_start='2019-03-04'
after_end='2019-03-27'

# Parameters for SAR
polarization = "VH"
pass_direction = "DESCENDING"
difference_threshold = 1.25
# # These orbits need to be change. Are they necessary given country bounds?
# relative_orbits = [63, 165, 92]

In [140]:
# SAR DATA SELECTION
collection = (
    ee.ImageCollection("COPERNICUS/S1_GRD")
    .filter(ee.Filter.eq("instrumentMode", "IW"))
    .filter(ee.Filter.listContains("transmitterReceiverPolarisation", polarization))
    .filter(ee.Filter.eq("orbitProperties_pass", pass_direction))
    .filter(ee.Filter.eq("resolution_meters", 10))
    .filterBounds(aoi)
    .select(polarization)
)

# Select images by predefined dates and get the median values of the 'before' image
before_collection = collection.filterDate(before_start, before_end)
before_med = ee.Image(before_collection.reduce(ee.Reducer.median()))
after_collection = collection.filterDate(after_start, after_end)

extentGeom = after_collection.geometry()

# Extract date from meta data
def dates(imgcol):
    range = imgcol.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
    message = (
        ee.String('from ')
        .cat(ee.Date(range.get('min')).format('YYYY-MM-dd'))
        .cat(' to ')
        .cat(ee.Date(range.get('max')).format('YYYY-MM-dd'))
    )
    return message

# print dates of before images to console
before_count = before_collection.size()
# print(ee.String('Tiles selected: Before Flood ({})'.format(before_count.getInfo()))
print('Tiles selected: Before Flood ({})'.format(before_count.getInfo()))
print(dates(before_collection).getInfo())

# print dates of after images to console
after_count = before_collection.size()
print('Tiles selected: After Flood  ({})'.format(after_count.getInfo()))
print(dates(after_collection).getInfo())

# Create a mosaic of selected tiles and clip to study area
before = before_med.clip(aoi)
after = after_collection.mosaic().clip(aoi)

# Apply reduce the radar speckle by smoothing  
smoothing_radius = 50
before_filtered = before.focal_mean(smoothing_radius, 'circle', 'meters')
after_filtered = after.focal_mean(smoothing_radius, 'circle', 'meters')

Tiles selected: Before Flood (186)
from 2019-01-01 to 2019-02-26
Tiles selected: After Flood  (186)
from 2019-03-04 to 2019-03-26


In [141]:
#------------------------------- FLOOD EXTENT CALCULATION -------------------------------//

# Calculate the difference between the before and after images
difference = after_filtered.divide(before_filtered)

# Apply the predefined difference-threshold and create the flood extent mask 
threshold = difference_threshold
difference_binary = difference.gt(threshold)

# Refine flood result using additional datasets
      
# Include JRC layer on surface water seasonality to mask flood pixels from areas
# of "permanent" water (where there is water > 10 months of the year)
surfwater = surfwater.select('seasonality')
swater_mask = surfwater.gte(10).updateMask(surfwater.gte(10))

#Flooded layer where perennial water bodies (water > 10 mo/yr) is assigned a 0 value
flooded_mask = difference_binary.where(swater_mask,0)
# final flooded area without pixels in perennial waterbodies
flooded = flooded_mask.updateMask(flooded_mask)

# Compute connectivity of pixels to eliminate those connected to 8 or fewer neighbours
# This operation reduces noise of the flood extent product 
connections = flooded.connectedPixelCount()    
flooded = flooded.updateMask(connections.gte(8))

# Mask out areas with more than 10 percent slope using a Digital Elevation Model 
terrain = ee.Algorithms.Terrain(DEM)
slope = terrain.select('slope')
flooded = flooded.updateMask(slope.lt(10))

In [142]:
#------------------------------- AFFECTED POPULATION -------------------------------#

# Sort population image collection by year, then mosaic so only latest pixel left
pop2019 = WorldPop.sort('year', True).mosaic()
# Mask to flooded area
affectPop = pop2019.mask(flooded)

# Convert flood raster to polygons
flooded_vec = flooded.reduceToVectors(
  scale= 10,
  geometryType ='polygon',
  geometry = aoi,
  eightConnected = False,
  bestEffort =True,
  tileScale =2,
)


# Summary statistics for the affected population in Mozambique.  
affectStat = affectPop.reduceRegion(
  reducer = ee.Reducer.sum(),
  geometry = flooded_vec.geometry(),
  scale = 100,
)

#sumAffectPop = affectStat.get('population')[0]
print(
    'The affected population is calculated to be: ' \
    + str(int(affectStat.get('population').getInfo()))
)

The affected population is calculated to be: 123809


In [143]:
### Set up Folium for interactive mapping ###
import folium
from folium import plugins

# Add custom basemaps to folium.  Taken from here: 
# https://colab.research.google.com/github/giswqs/qgis-earthengine-examples/blob/master/Folium/ee-api-folium-setup.ipynb
basemaps = {
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    )
}

# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
#----------------- Add layers to Map--------------------------

# Create a folium map object.
my_map = folium.Map(location=[-25, 32], zoom_start=6)

# Add custom basemaps
basemaps['Google Satellite'].add_to(my_map)


my_map.add_ee_layer(aoi, {'palette': 'red'}, 'Mozambique')

my_map.add_ee_layer(extentGeom, {'min': 0,  'max': 4000}, 'Satellite extent')

my_map.add_ee_layer(difference, {'min':0, 'max':2}, "Difference Layer")

my_map.add_ee_layer(flooded, {'palette': "0000FF"}, 'Flooded Area')

my_map.add_ee_layer(affectPop, {'palette': ["fff7ec","990000"]}, 'Affected Population')


# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)